In this notebook we work on the problem of co-registering between CaSSIS, CTX and CRISM sensors, in preparation for correcting image misalignment as part of the CaSSIS spectral parameters project.

We want the co-registered products to be in the same coordinate reference system (CRS), so we will also use the `pyproj` library to work with the data.

# Overview

1. Load in the CTX base map as an xarray
2. Load in the CRISM VNA map as an xarray
3. Co-register using Geowombat and ACROSICS
4. Apply the co-registration warp to another dataset

Please note that in addition to these Python packages, `ghostscript` version `9.56.1` is a requirement, to ensure PyGMT transparency works correctly (this may be an issue for MacOS only).

In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import planetary_arosics as paros
from pathlib import Path
import pygmt
import rasterio as rio

# Reading in the CTX Dataset

In the `example_scenes` directory is a copy of the Jezero Crater orthorectified CTX 5 m/px resolution mosaic, that can be found here:

https://asc-pds-services.s3.us-west-2.amazonaws.com/mosaic/mars2020_trn/CTX/ScienceInvestigationMaps_JPL/M20_JezeroCrater_CTXortho_mosaic_5m.tif

The data is a GeoTIFF, with a coordinate reference system embedded in the file.

We read in this mosaic. This will be the target scene, that we will be co-registering other scenes to.

In [ ]:
ctx_mosaic_path = Path('.', 'input_products', 'ctx', 'orthomosaic', 'm20_jezerocrater', 'M20_JezeroCrater_CTXortho_mosaic_5m.tif')
ctx_mosaic_lbl = Path('.', 'input_products', 'ctx', 'orthomosaic', 'm20_jezerocrater', 'M20_JezeroCrater_CTXortho_mosaic_5m.xml')

The full mosaic, within it's coordinate reference system (CRS), covers the following region, that we display using PyGMT.

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_path,
    shading=False,
    cmap='gray',
    # projection='M10c',
    frame=['xaf','yaf', '+tJezero Crater CTX Orthomosaic 5 m/px'],
)
fig.show()

To align this CTX map with the CRISM map product, we need to convert the co-ordinate reference system to a geographic co-ordinate reference system, i.e. in units of Lat/Lon.

We achieve this using the following function, that accesses the geographic CRS (GRS) metadata, and performs the conversion.

We apply this to the CTX GeoTIFF. Note that this produces a new data product, that we attach the GRS suffix to, and save in the same directory as the source product.

In [ ]:
ctx_mosaic_grs_path = paros.geotiff_2_geo_crs(ctx_mosaic_path)

The product is now displayed with axes of Latitude and Longitude.

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    shading=False,
    cmap='gray',
    projection='M15c',
    frame=['xaf','yaf', '+tJezero Crater CTX Orthomosaic 5 m/px'],
)
fig.show()

# Reading in the CaSSIS Data

In [ ]:
cassis_eg1_path = Path('.', 'input_products', 'cassis', 'cubes', 'MY35_014234_019', 'MY35_014234_019_1.cubeit.cub')

For CaSSIS, we need a new function that produces, from a PDS readable ISIS Cube, a set of 24-bit colour browse products in the same projection, that can then be read and converted to geotiff.

In [ ]:
from typing import Dict, Literal

In [ ]:
from osgeo import gdal, gdal_array

In [ ]:
import numpy as np

In [ ]:
BROWSE_PRODUCT_DICT = {
    'BLU': [1],
    'PAN': [2,2,2],
    'RED': [3],
    'NIR': [4],
    'NPB': [1, 2, 4],
    'NRB': [1, 3, 4],
    'NRP': [2, 3, 4],
    'RPB': [1, 2, 3]
}

def cassis_browse_product_generator(
        cassis_cube: Path,
        browse_product: Literal['all', 
                                'BLU', 'PAN', 'RED', 'NIR', 
                                'NPB', 'NRB', 'NRP', 'RPB',
                                'RGB']='all',
        stretch_params: Dict={'p_lo': 0.0001, 'p_hi': 0.9995}) -> Path:
    """Generate a CaSSIS browse product from a CaSSIS cube file.

    :param cassis_cube: Path to the CaSSIS cube file.
    :return: Path to the generated CaSSIS browse product GeoTIFF file.
    """
    # first we read the file in with rio to get the noData value
    with rio.open(cassis_cube) as src:
        noData = src.read().min()

    gdal.AllRegister()

    geo_tiff_out = paros.output_path(cassis_cube, suffix=f'_{browse_product}.tif')

    browse_bands = BROWSE_PRODUCT_DICT[browse_product]

    # apply stretches - credit to Adriano Tullo PANNCO for this code: https://github.com/adritullo/PANCO/blob/main/Coregistration/Cassis_Reproject.py

    inputinfo = gdal_array.LoadFile(str(cassis_cube), band_list=browse_bands).astype(float)
    inputinfo[inputinfo <= 0] = np.nan
    minvals = []
    maxvals = []
    # if inputinfo has 3 dims:
    if len(inputinfo.shape) != 3:
        # reshape to 3 dims
        inputinfo = inputinfo.reshape((1, inputinfo.shape[0], inputinfo.shape[1]))
    n_bands = inputinfo.shape[0]
    for band in range(n_bands):
        minval = np.nanquantile(inputinfo[band], stretch_params['p_lo'])
        maxval = np.nanquantile(inputinfo[band], stretch_params['p_hi'])
        print(f"Image range: {minval} - {maxval}")
        minvals.append(minval)
        maxvals.append(maxval)
    inputinfo = None

    gdal.Translate(
        str(geo_tiff_out),
        str(cassis_cube),
        format='GTiff',                
        bandList=browse_bands,        
        noData=255,
        outputType=gdal.GDT_Byte,
        scaleParams=[[minvals[band], maxvals[band], 0, 255] for band in range(n_bands)],
        creationOptions=['COMPRESS=LZW']
    )
    
    cassis_browse_gtiff = geo_tiff_out

    return cassis_browse_gtiff

In [ ]:
cassis_pan_gtiff_path = cassis_browse_product_generator(cassis_eg1_path, browse_product='PAN')

The CaSSIS cube is 4 channel, so when we display it we'll need to choose a band to show.

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=cassis_pan_gtiff_path,
    shading=False,    
    verbose=True,
    cmap='gray',
    nan_transparent="white",
    # projection='10c',
    frame=['xaf','yaf', '+tJezero Crater CaSSIS Example 4 m/px'],
)
fig.show()

In [ ]:
cassis_pan_grs_path = paros.geotiff_2_geo_crs(cassis_pan_gtiff_path)

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=cassis_pan_grs_path,    
    cmap='gray',
    nan_transparent="white",
    verbose=True,
    projection='M10c',
    frame=['xaf','yaf', '+tJezero Crater CaSSIS Example 4 m/px'],
)
fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    shading=False,
    cmap='gray',
    projection='M10c',
    frame=['xaf','yaf', '+tJezero Crater CTX Orthomosaic 5 m/px'],
)
fig.grdimage(
    grid=cassis_pan_grs_path,
    nan_transparent="white",  
    cmap='gray',
    projection='M10c',
    transparency=50,
    # frame=['xaf','yaf', '+t \n Jezero Crater CaSSIS Example 4 m/px'],
)
fig.show()

# Showing the Image Misalignment

We can focus on the Jezero Delta by providing an approximate bounding square in the default CRS.

We get this boundary out of the CaSSIS data product, by reading the GeoTIFF with `rasterio`.

In [ ]:
cassis_eg1_region = paros.get_raster_grs_bounds(cassis_pan_grs_path)

In [ ]:
cassis_eg1_region

Now displaying this region, we can also overlay the CaSSIS product, with transparency to assess the mis-alignment.

This image is shown at the end of the notebook, too avoid a problem of crashing.

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    shading=False,
    region=cassis_eg1_region,
    projection="M8c",
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Crater CTX Orthomosaic 5 m/px'],
)
fig.grdimage(
    grid=cassis_pan_grs_path,
    nan_transparent="white", 
    region=cassis_eg1_region,  
    projection="M8c",      
    cmap='gray',
    transparency=50,
    frame=['xaf','yaf', '+t \n Jezero Crater CaSSIS Example 4 m/px'],
)
fig.show()

# Co-Registration with AROSICS

Here we attempt to interface with AROSICS directly, and do not use geowombat.

We have a problem where we need both datasets to use the same CRS.

Here we use the pyproj CRS library to take the CRS of the CTX basemap, and apply this to the CaSSIS image.

In [ ]:
cassis_pan_grs_ctx_path = paros.inherit_crs(
                            ctx_mosaic_grs_path,
                            cassis_pan_grs_path,
                            'ctx-crs'
                        )

Now we can check the CRS of the newly created geotiff of the CaSSIS scene.

In [ ]:
ctx_mosaic = rio.open(ctx_mosaic_grs_path)
cassis_pan = rio.open(cassis_pan_grs_ctx_path)
ctx_mosaic.crs == cassis_pan.crs

## Global Co-Registration with AROSICS

First we will experiment with the simpler co-registration tool, COREG.

This finds an X/Y shift between a target and reference image.

Let's make a destination path for the co-registered CRISM image.

Now we pass the reference and target images to the COREG function.

For CaSSIS images, 255 is the no-data value that we set here.

We have found that a smaller window is more reliable, so we set this to 64 x 64 pixels.

This example image requires a shift greater than the default max_shift of 5. Here we set this to 20 pixels.

In [ ]:
ref_path = ctx_mosaic_grs_path
tgt_path = cassis_pan_grs_ctx_path

In [ ]:
cassis_pan_grs_ctx_path

In [ ]:
cassis_pan_grs_ctx_coreg_path, CRG = paros.arosics_global_coreg(ref_path, tgt_path)

### Masking the Target Image

CaSSIS footprints are long, and may cover a region of highly varying topography. CaSSIS images are also not sun-synchronous, so the images are not good approximations of an orthorectified map product. This means that pixel-shifting may only optimize co-registration of small subsets of the product. We can use masking in via rasterio to select specific regions of the CaSSIS product to co-register.

Here we experiment with selecting the Jezero Delta region to optimize the co-registration over.

In [ ]:
jezero_delta_region = '77.21/77.47/18.40/18.60'

In [ ]:
bbox = paros.str_bounds_to_numpy(jezero_delta_region)

In [ ]:
from geoarray.baseclasses import GeoArray
from rasterio.windows import from_bounds

In [ ]:
with rio.open(tgt_path) as ds:
    transform=ds.transform

In [ ]:
nodata_val = 255
# Read raster metadata
with rio.open(tgt_path) as ds:
    # Compute the pixel window covering the bounding box
    window = from_bounds(*bbox, transform=ds.transform)
    window = window.round_offsets().round_lengths()

    # Make a base boolean array: True = outside bbox
    base_mask = np.ones((ds.height, ds.width), dtype=bool)

    # Set inside bbox pixels to False
    base_mask[
        window.row_off : window.row_off + window.height,
        window.col_off : window.col_off + window.width
    ] = False

    # Optionally, represent masked vs. unmasked with nodata_val
    # (e.g., if users want 0/1 or specific class codes)
    mask_arr = base_mask.astype(type(nodata_val))
    mask_arr[~base_mask] = nodata_val

    # Create a GeoArray from this array & georeferencing info
    mask_ga = GeoArray(
        mask_arr,
        geotransform=ds.transform.to_gdal(),
        projection=ds.crs.to_wkt() if ds.crs else None
        )

In [ ]:
cassis_pan_grs_ctx_coreg_path, CRG = paros.arosics_global_coreg(ref_path, tgt_path, tgt_mask_bbox=jezero_delta_region)

## Local Co-Registration with AROSICS

In [ ]:
ref_path = ctx_mosaic_grs_path
tgt_path = cassis_pan_grs_ctx_coreg_path

In [ ]:
cassis_pan_grs_ctx_local_coreg_path, CRL = paros.arosics_local_coreg(ref_path, tgt_path, grid_res=8)

In [ ]:
cassis_pan_grs_ctx_local_coreg_path

## Applying the Shifts to a CaSSIS Browse Product

Let's now check that we can use the shifts found here can be applied to a CaSSIS browse product.

Let's generate an NPB browse product.

In [ ]:
cassis_npb_gtiff_path = cassis_browse_product_generator(cassis_eg1_path, browse_product='NPB')

We also need to convert this product to the geographic coordinate reference system.

In [ ]:
cassis_npb_grs_path = paros.geotiff_2_geo_crs(cassis_npb_gtiff_path)

We need to inherit the CTX CRS information.

In [ ]:
cassis_npb_grs_ctx_path = paros.inherit_crs(
    ctx_mosaic_grs_path,
    cassis_npb_grs_path,
    'ctx-crs'
)

In [ ]:
cassis_npb_grs_ctx_path

Now we can apply the global and local co-registration shifts found for the PAN product.

In [ ]:
cassis_npb_grs_ctx_crg_path = paros.apply_coreg(cassis_npb_grs_ctx_path, global_crg=CRG, local_crg=None)

In [ ]:
cassis_npb_grs_ctx_crl_path = paros.apply_coreg(cassis_npb_grs_ctx_path, global_crg=CRG, local_crg=CRL)

# Checking the results.

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=cassis_eg1_region,
    projection="M8c",
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ CaSSIS PAN 4 m/px with 5 m/px CTX @^ Prior to Coregistration'],
)

fig.grdimage(
    grid=cassis_pan_grs_ctx_path,
    region=cassis_eg1_region,
    projection="M8c",
    nan_transparent="white",
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=cassis_eg1_region,
    projection="M8c",
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ CaSSIS PAN 4 m/px with 5 m/px CTX @^ Global Co-Registration'],
)

fig.grdimage(
    grid=cassis_npb_grs_ctx_crg_path,
    region=cassis_eg1_region,
    nan_transparent="white",
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=cassis_eg1_region,
    projection="M8c",
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ region=cassis_eg1_region, with 5 m/px CTX @^ Global & Local Co-Registration'],
)

fig.grdimage(
    grid=cassis_pan_grs_ctx_local_coreg_path,
    region=cassis_eg1_region,
    projection="M8c",
    nan_transparent="white",
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=cassis_eg1_region,
    projection="M8c",
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ CRISM MTRDR TRU 18 m/px with 5 m/px CTX @^ Global & Local Co-Registration'],
)

fig.grdimage(
    grid=cassis_npb_grs_ctx_crl_path,
    region=cassis_eg1_region,
    projection="M8c",
    nan_transparent="white",
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

OK, this completes the goal of co-registration of geo-reference CRISM to CTX data.